In [1]:
import pandas as pd
import numpy as np

features2 = ['SLHC', 'HST', 'HCT', 'MCV', 'MCH', 'MCHC', 'RDWCV', 'SLTC', 'SLBC', 'FE', 'FERRITIN', 'HBA1', 'HBA2', 'HBE', 'HBF']
features = ['SLHC', 'HST', 'HCT', 'MCV', 'MCH', 'MCHC', 'RDWCV', 'SLTC', 'SLBC']

In [2]:
df1 = pd.read_csv("../datasets/processed_data.csv", low_memory=False)
df1.PID = df1.PID.astype(str)
df1.set_index("PID", inplace=True)
feature_df = df1[features]
feature_df = feature_df.apply(pd.to_numeric, errors='coerce', downcast='float')
df1 = df1.loc[feature_df.dropna().index][features2 + ['sheet']]
df1 = df1.assign(alpha=0, beta=0)
condition = df1.sheet.isin(['alpha', 'fe_alpha', 'alpha_beta'])
df1['alpha'] = condition.astype(int)

condition = df1.sheet.isin(['beta', 'fe_beta', 'alpha_beta'])
df1['beta'] = condition.astype(int)
# df1 = df1.assign(sheet='1803')
print(df1[df1.sheet == 'normal'].MCV.min())

85.0


In [3]:
def get_df(file_path, sheet_type):
    df = pd.read_csv(file_path)
    df.set_index('PID', inplace=True)

    return df.assign(sheet=sheet_type)

df2 = get_df("../datasets/0504.csv", '0504')
df3 = get_df("../datasets/1904.csv", '1904')
df4 = get_df("../datasets/2704.csv", '2704')
df5 = get_df("../datasets/2804.csv", '2804')
df6 = get_df("../datasets/2904.csv", '2904')

df_arr = [df1, df2, df3, df4, df5, df6]
for e in df_arr:
    print(e.shape)

(19108, 18)
(1671, 18)
(663, 18)
(772, 18)
(664, 18)
(264, 18)


In [5]:
pd.concat(df_arr).to_csv("tong_hop1.csv")

In [54]:
def normal_hc(df):
    conditions = (df.alpha + df.beta == 0) & (df.MCV >= 85) &(df.MCH >= 28)
    idxes = conditions.index
    return conditions.sum(), idxes, conditions

def both_ab(df):
    conditions = (df.alpha + df.beta == 2)
    idxes = conditions.index
    return conditions.sum(), idxes, conditions

def alpha(df):
    conditions = df.alpha == 1
    idxes = conditions.index
    return conditions.sum(), idxes, conditions

def beta(df):
    conditions = df.beta == 1
    idxes = conditions.index
    return conditions.sum(), idxes, conditions

def lack_fe(df):
    conditions = (~df.FERRITIN.isna()) & (df.FERRITIN < 30)
    df_fe = df[conditions]
    
    alpha_num, alpha_idx, alpha_condition = alpha(df_fe)
    beta_num, beta_idx, beta_condition = beta(df_fe)
    
    print('fe_alpha', alpha_num)
    print("fe_beta", beta_num)
    
    df_fe2 = df_fe[~(alpha_condition | beta_condition)]
    
    return df_fe2.shape[0], df_fe2.index, conditions
    
    
    

def statistic(df):
    df[features2] =  df[features2].apply(pd.to_numeric, errors='coerce', downcast='float')
    normal_num, normal_idx, normal_conditions = normal_hc(df)
    
    dfa = df[~normal_conditions]
    ab_num, ab_idx, ab_conditions = both_ab(dfa)
    
    dfb = dfa[~ab_conditions]
    fe_num, fe_idx, fe_conditions = lack_fe(dfb)
    
    dfc = dfb[~fe_conditions]
    alpha_num, alpha_idx, alpha_condition = alpha(dfc)
    beta_num, beta_idx, beta_condition = beta(dfc)
    
    print('normal', normal_num)
    print("alpha_beta", ab_num)
    print('normal_fe', fe_num)
    print('alpha', alpha_num)
    print("beta", beta_num)
    
    
# for df in df_arr:
#     print(df.shape)
#     statistic(df)
#     print("----------------------------")

In [55]:
df = pd.concat([df1, df2, df3, df4, df5, df6])

In [56]:
df.shape

(23142, 18)

In [57]:
statistic(df)

fe_alpha 775
fe_beta 303
normal 10164
alpha_beta 1855
normal_fe 1109
alpha 4971
beta 2851


In [130]:
def get_normal(df):
    conditions = (df.alpha + df.beta == 0) & (df.MCV >= 85) &(df.MCH >= 28)
    return df[conditions]


def get_alpha_beta(df):
    conditions = (df.alpha + df.beta == 2)
    return df[conditions]

def get_alpha(df):
    conditions1 = ~((df.alpha + df.beta == 0) & (df.MCV >= 85) &(df.MCH >= 28))
    conditions2 = (df.alpha == 1) & (df.beta == 0)
    conditions3 = (~df.FERRITIN.isna()) & (~df.FE.isna()) & (df.FERRITIN < 30)
    return df[conditions1 & conditions2 & ~conditions3 ]

def get_beta(df):
    conditions1 = ~((df.alpha + df.beta == 0) & (df.MCV >= 85) &(df.MCH >= 28))
    conditions2 = (df.alpha == 0) & (df.beta == 1)
    conditions3 = (~df.FERRITIN.isna()) & (~df.FE.isna()) & (df.FERRITIN < 30)
    return df[conditions1 & conditions2 & ~conditions3 ]
    

def get_normal_fe(df):
    conditions = (~df.FERRITIN.isna()) & (~df.FE.isna()) & (df.FERRITIN < 30) & (df.alpha + df.beta == 0)
    print(df.alpha.unique(), df.beta.unique())
    return df[conditions]

def negative_unnormal(df):
    conditions1 = ~((df.MCV >= 85) &(df.MCH >= 28))
    conditions2 = df.alpha + df.beta == 0
    return df[conditions1 & conditions2]

get_alpha(df).shape, get_beta(df).shape, negative_unnormal(df).shape

((4972, 18), (2851, 18), (2223, 18))

In [131]:
df.assign(thalas = (df.alpha + df.beta > 0).astype(int))

,SLHC,HST,HCT,MCV,MCH,MCHC,RDWCV,SLTC,SLBC,FE,FERRITIN,HBA1,HBA2,HBE,HBF,sheet,alpha,beta,thalas
PID,,,,,,,,,,,,,,,,,,,
97500602,3.02,81.0,0.293,97.099998,26.900000,277.0,17.000000,195.0,7.55,28.0,67.099998,96.000000,1.0,NaN,NaN,alpha,1,0,1
97220957,4.11,123.0,0.399,97.099998,29.900000,308.0,13.600000,214.0,9.56,NaN,NaN,98.300003,1.7,NaN,NaN,alpha,1,0,1
97231143,5.21,141.0,0.495,94.900002,27.100000,286.0,13.500000,339.0,6.54,NaN,NaN,97.800003,2.2,NaN,NaN,alpha,1,0,1
97221041,4.42,133.0,0.418,94.500000,30.200001,319.0,14.100000,165.0,8.13,NaN,NaN,97.800003,2.2,NaN,NaN,alpha,1,0,1
97220964,4.91,146.0,0.463,94.300003,29.700001,315.0,14.200000,270.0,8.18,19.6,1202.099976,97.900002,2.1,NaN,NaN,alpha,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15038717,4.04,107.0,0.341,84.500000,26.500000,314.0,13.400000,267.0,5.34,56.0,NaN,97.699997,2.3,NaN,NaN,2904,1,0,1
15032051,5.26,138.0,0.435,82.599998,26.100000,316.0,15.200000,177.0,16.65,12.5,353.100006,96.900002,3.1,NaN,NaN,2904,0,0,0
15030562,4.45,139.0,0.420,94.199997,31.100000,330.0,13.600000,273.0,4.36,15.0,98.000000,97.500000,2.5,NaN,NaN,2904,0,0,0


In [149]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, RandomForestRegressor

numerical_columns = ["SLHC","HST","HCT","MCV","MCH","MCHC","RDWCV","SLTC","SLBC", "FERRITIN","FE"]

def fit_model(model, df):
#     global numerical_columns
    
    df[numerical_columns].dropna()
    print(df.shape, df[numerical_columns].dropna().shape)
    X = df[numerical_columns]
    y = df['thalas']

    # model.partial_fit(X, y)
    model.fit(X, y)
    

def evaluate(model, df, desc=''):
#     global numerical_columns
    
    X = df[numerical_columns]
    y = df['thalas']
    
    preds = np.array(model.predict(X))
    print('---------------------------------------------------')
    print(f"evaluation results {desc}:\n")
    print(classification_report(y, np.where(preds>=0.5, 1, 0)))
    print('---------------------------------------------------')

    # mask = preds != y
    mask = (0.3 <= preds) & (preds <= 0.7)
    print(sum(mask))

    features_df = df[mask][numerical_columns].reset_index(drop=True)
    labels = preds[mask]
    labels = np.where(labels>=0.5, 1, 0)
    # print(labels)
    new_df = pd.concat([features_df, pd.DataFrame({'thalas': labels})], axis=1)

    return new_df, df[preds == y].reset_index(drop=True)

In [150]:
# model = RandomForestRegressor(n_estimators = 100, random_state=42,n_jobs=8)
# model = xgb.XGBClassifier(learning_rate=0.1, max_depth=10, n_estimators=100, n_jobs=-1, random_state=42,use_label_encoder=False, objective="binary:logistic")
model = GaussianNB()

In [161]:
# beta_df = get_alpha(df)
beta_df = get_beta(df)
alpha_df = negative_unnormal(df)

alpha_df.shape, beta_df.shape

alpha_df = alpha_df.assign(thalas=0)
beta_df = beta_df.assign(thalas=1)

main_df = pd.concat([alpha_df, beta_df])
dropted_df = main_df.loc[main_df[numerical_columns].dropna().index]
dropted_df = dropted_df.loc[dropted_df[numerical_columns].dropna().index]
train_df, test_df = train_test_split(dropted_df, test_size=0.2, stratify=dropted_df.thalas)

# alpha_df.shape, beta_df.shape, main_df.shape, dropted_df.shape, dropted_df[numerical_columns].dropna().shape

In [162]:
fit_model(model, train_df)
# train_df.info()

(3860, 19) (3787, 11)


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [141]:
_ = evaluate(model, test_df)

---------------------------------------------------
evaluation results :

              precision    recall  f1-score   support

           0       0.92      0.80      0.85       340
           1       0.84      0.93      0.88       382

    accuracy                           0.87       722
   macro avg       0.88      0.87      0.87       722
weighted avg       0.88      0.87      0.87       722

---------------------------------------------------
0
